In [ ]:
# ! pip install nbformat>=4.2.0

In [23]:
import os
import sys
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

absolute_parent = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(absolute_parent)

from tdf_utility.trading.nse_api import NSE_API, get_nse_india_vix, get_nse_market_status_daily

nse_api = NSE_API()

def load_graph_data_to_df(object: dict):
    identifier = object.get('data').get('identifier')
    # name = objecct.get('data').get('name')
    grapth_data = object.get('data').get('grapthData')
    df = pd.DataFrame(grapth_data, columns=['Timestamp', 'Price', 'Code'])
    df['Date'] = pd.to_datetime(df['Timestamp'], unit='ms')
    return df, identifier

In [24]:
# 1. Setup the source data
nifty_50_1y_historical_data = nse_api._get_data("api/NextApi/apiClient/historicalGraph?functionName=getIndexChart&&index=NIFTY%2050&flag=1Y")

india_vix_1y_historical_data = get_nse_india_vix()
india_vix_1y_historical_data['date'] = pd.to_datetime(india_vix_1y_historical_data['date'], format='%d-%b-%Y')

gold_historical_data = nse_api._get_data("api/historical-spot-price?symbol=GOLD&fromDate=30-12-2024&toDate=30-12-2025")

gold_historical_df = pd.DataFrame(gold_historical_data['data']) \
                                    .loc[:, ['UpdatedDate', 'SpotPrice2']] \
                                    .rename(columns={'UpdatedDate': 'Date', 'SpotPrice2': 'Gold 10gm'})

gold_historical_df['Date'] = pd.to_datetime(gold_historical_df['Date'], format='%d-%b-%Y')

In [25]:
# 2. process the source data
graph_df, graph_identifier = load_graph_data_to_df(nifty_50_1y_historical_data)

viz_df = pd.merge(graph_df.loc[:, ['Date', 'Price']].rename(columns={'Price': 'Nifty 50'}),
                  india_vix_1y_historical_data[['date', 'close']].rename(columns={'date': 'Date', 'close': 'India VIX'}),
                  on='Date', how='left')

viz_df = pd.merge(viz_df, gold_historical_df, on='Date', how='left')

In [11]:
# 3. Visualize the data


# 3.1. Create Figure with Secondary Y-Axis
# This function creates the container that allows two Y-axes
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 3.2. Add Trace 1: Nifty 50 (Left Axis)
fig.add_trace(
    go.Scatter(
        x=viz_df['Date'], 
        y=viz_df['Nifty 50'], 
        name="Nifty 50",
        line=dict(color='#00C805', width=2)
    ),
    secondary_y=False, # Assign to primary (left) axis
)

fig.add_trace(
    go.Scatter(
        x=viz_df['Date'], 
        y=viz_df['Gold 10gm'], 
        name="Gold 10gm",
        line=dict(color="#C8AA00", width=2)
    ),
    secondary_y=False, # Assign to primary (left) axis
)

# 3.3. Add Trace 2: India VIX (Right Axis)
fig.add_trace(
    go.Scatter(
        x=viz_df['Date'], 
        y=viz_df['India VIX'], 
        name="India VIX",
        line=dict(color='#FF5733', width=2) # Red, dotted line
    ),
    secondary_y=True, # Assign to secondary (right) axis
)

# 3.4. Customize Layout
fig.update_layout(
    title_text="Price vs Volatility",
    template="plotly_dark",
    hovermode="x unified"
)

# 3.5. Set axis titles
fig.update_yaxes(title_text="<b>Nifty Price</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>VIX Level</b>", secondary_y=True)

fig.update_layout(
    title=f"Nifty50 and India VIX Trend",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark", # Use 'plotly_white' for light theme
    hovermode="x unified",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

fig.show()

In [12]:
# graph_df.loc[:, ['Date', 'Price']].rename(columns={'Price': 'Nifty 50'}).dtypes
# india_vix_1y_historical_data[['date', 'close']].rename(columns={'date': 'Date', 'close': 'India VIX'})

In [3]:
import plotly.express as px
df = px.data.stocks()

In [33]:
viz_df.iloc[-1]["Nifty 50"]

np.float64(25938.85)

In [29]:
latest_data

Date         2025-12-30 00:00:00
Nifty 50                25938.85
India VIX                   9.68
Gold 10gm                 134158
Name: 248, dtype: object

In [14]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- 1. Data Simulation (Matching your provided start/end points) ---
def generate_market_data():
    dates = pd.date_range(start="2024-12-30", periods=250, freq='B') # Business days
    
    # Linear interpolation + Noise for realistic look
    nifty = np.linspace(23644, 25938, len(dates)) + np.random.normal(0, 150, len(dates))
    vix = np.linspace(13.97, 9.68, len(dates)) + np.random.normal(0, 0.5, len(dates))
    gold = np.linspace(75953, 134158, len(dates)) + np.random.normal(0, 500, len(dates))
    
    return pd.DataFrame({'Date': dates, 'Nifty 50': nifty, 'India VIX': vix, 'Gold': gold})

df = generate_market_data()

# --- 2. Streamlit Setup ---


# --- 3. Build Stacked Subplots ---
# rows=3, cols=1 creates a vertical stack
fig = make_subplots(
    rows=3, cols=1, 
    shared_xaxes=True, # vital for comparing timeframes
    vertical_spacing=0.05,
    subplot_titles=("Nifty 50 (Equity)", "India VIX (Volatility)", "Gold 10gm (Commodity)"),
    specs=[[{"secondary_y": False}], [{"secondary_y": False}], [{"secondary_y": False}]]
)

# Row 1: Nifty 50
fig.add_trace(go.Scatter(
    x=df['Date'], y=df['Nifty 50'],
    name="Nifty 50", mode='lines',
    line=dict(color='#00E396', width=2) # Green
), row=1, col=1)

# Row 2: India VIX (Inverse indicator, often red/orange)
fig.add_trace(go.Scatter(
    x=df['Date'], y=df['India VIX'],
    name="India VIX", mode='lines',
    line=dict(color='#FF4560', width=2) # Red
), row=2, col=1)

# Row 3: Gold (Safe haven, often yellow/gold)
fig.add_trace(go.Scatter(
    x=df['Date'], y=df['Gold'],
    name="Gold", mode='lines',
    line=dict(color='#FEB019', width=2) # Gold color
), row=3, col=1)

# --- 4. Professional Styling ---
fig.update_layout(
    height=800, # Taller chart to accommodate 3 panels
    template="plotly_dark",
    hovermode="x unified", # Shows all 3 values in one tooltip when hovering a date
    showlegend=False, # Hide legend to save space (titles are sufficient)
    margin=dict(t=50, b=50, l=50, r=50)
)

# Add Range Slider only to the bottom plot, but it controls all
fig.update_xaxes(matches='x') 


In [16]:
market_status = get_nse_market_status_daily()

In [18]:
market_status

,nifty50_close,nifty50_change,nifty50_perchange,nifty50_status,giftnifty_close,giftnifty_change,giftnifty_perchange,marketcapin_trdollars,marketcapin_laccr_rupees,as_of_date,current_date_time
0,25938.85,-3.22,-0.01,CLOSE,None,None,None,5.22,469.187764,30-Dec-2025 15:30,None


In [20]:

import pandas as pd

# 1. Setup Dummy Data (Matching your previous structure)
data = {
    "Date": ["2025-12-29", "2025-12-30"],
    "Nifty 50": [25942.10, 25938.85],  # Price dropped slightly
    "India VIX": [9.72, 9.68]          # Volatility dropped slightly
}
df = pd.DataFrame(data)

# 2. Calculation Logic
# Get the latest data (Last row)
latest_data = df.iloc[-1]
# Get the previous data (Second to last row)
prev_data = df.iloc[-2]

# Calculate Changes (Deltas)
nifty_change = latest_data["Nifty 50"] - prev_data["Nifty 50"]
vix_change = latest_data["India VIX"] - prev_data["India VIX"]

# 3. Create Columns
col1, col2 = st.columns(2)

# 4. Render Metrics
# f"{value:.2f}" formats the number to 2 decimal places

print(f"{latest_data['Nifty 50']:.2f}", f"{nifty_change:.2f}")

print(f"{latest_data['India VIX']:.2f}", f"{vix_change:.2f}")

2025-12-30 20:19:34.777 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-30 20:19:34.778 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-30 20:19:34.779 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


25938.85 -3.25
9.68 -0.04


In [22]:
df.iloc[-1]["Nifty 50"]

np.float64(25938.85)